In [1]:
from langchain_community.document_loaders import FireCrawlLoader
from langchain_community.document_loaders import Docx2txtLoader
import threading

from langchain_core.pydantic_v1 import BaseModel, Field, validator, root_validator
# Load HTML
from langchain_community.document_loaders import FireCrawlLoader
from typing import List
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
import os
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI
from typing import List



In [2]:
from dotenv import  load_dotenv

In [3]:
load_dotenv(dotenv_path='../src/.env')

True

In [4]:
def get_html(url,mode='scrape',ignore_pages=[],max_pages=4):
    if mode == 'scrape':
        loader = FireCrawlLoader(
        api_key=os.environ['FIRECRAWL_API_KEY'], url=url, mode=mode
            )
    else:
        crawl_params = {
                'crawlerOptions': {
                    'excludes': [
                        'blog/*', 'login/*', 'account/*', 'user/*', 'profile/*',
                        'admin/*', 'dashboard/*', 'search/*', 'filter/*',
                        'checkout/*', 'payment/*', 'cart/*', 'css/*', 'js/*',
                        'images/*', 'assets/*', 'temp/*', 'under-construction/*',
                        'api/*', '404', '500', 'downloads/*', 'files/*', 'pdfs/*',
                        'archive/*', 'old/*', 'version/*', 'forum/*', 'comments/*',
                        'reviews/*', 'external/*', 'outbound/*', 'product/*', 'shop/*',
                        'category/*', 'promo/*', 'deals/*', 'offers/*', 'help/*',
                        'support/*', 'news/*', 'press/*', 'events/*',
                        'calendar/*', 'subscribe/*', 'signup/*', 'test/*', 'staging/*',
                        'wp-admin/*', 'backend/*', 'admin-panel/*', 'management/*'
                    ],
                    'includes': [], # leave empty for all pages
                    'limit': max_pages,
                }
            }

        loader = FireCrawlLoader(
        api_key=os.environ['FIRECRAWL_API_KEY'], url=url, mode=mode,
        params=crawl_params
            ) 
    docs = loader.load()
    # print(docs)
    return docs

In [6]:
!pip install firecrawl-py

  Using cached firecrawl_py-0.0.20-py3-none-any.whl.metadata (6.3 kB)
Using cached firecrawl_py-0.0.20-py3-none-any.whl (9.5 kB)


In [10]:
html_content = get_html("https://thelondondentalcentre.co.uk/")

In [13]:
html_content[0].page_content

'By using this website, you agree to our use of cookies. We use cookies to provide you with a great experience and to help our website run effectively.\n\nOK\n\n[0](/cart)\n\n[](https://thelondondentalcentre.co.uk/more-services/0-ldc-finance)\n\nInterest-Free Finance Available\n\n\n\n[![London Dental Centre](https://thelondondentalcentre.co.uk/images.squarespace-cdn.com/content/v1/5d7bafb2d4f5e4735f21b5cc/8c5f5d2f-957b-403a-9403-9bec057add03/Web+elements+%281920+x+500+px%29.png?format=1500w)](/)\n\n[About Us](/about-us)\n\n[Treatments](/treatments)\n\n[Fees](/fees)\n\n[020 3667 7070](tel:02036677070)\n\n[Book Online Now](https://uk.dentalhub.online/soe/new/London%20Dental%20Centre?pid=UKLZC01)\n\nOpen Menu Close Menu\n\n[![London Dental Centre](https://thelondondentalcentre.co.uk/images.squarespace-cdn.com/content/v1/5d7bafb2d4f5e4735f21b5cc/8c5f5d2f-957b-403a-9403-9bec057add03/Web+elements+%281920+x+500+px%29.png?format=1500w)](/)\n\n[About Us](/about-us)\n\n[Treatments](/treatments)\

In [9]:
# !pip install markdown

In [14]:
import markdown


In [15]:
html = markdown.markdown(html_content[0].page_content)


In [16]:
html

'<p>By using this website, you agree to our use of cookies. We use cookies to provide you with a great experience and to help our website run effectively.</p>\n<p>OK</p>\n<p><a href="/cart">0</a></p>\n<p><a href="https://thelondondentalcentre.co.uk/more-services/0-ldc-finance"></a></p>\n<p>Interest-Free Finance Available</p>\n<p><a href="/"><img alt="London Dental Centre" src="https://thelondondentalcentre.co.uk/images.squarespace-cdn.com/content/v1/5d7bafb2d4f5e4735f21b5cc/8c5f5d2f-957b-403a-9403-9bec057add03/Web+elements+%281920+x+500+px%29.png?format=1500w" /></a></p>\n<p><a href="/about-us">About Us</a></p>\n<p><a href="/treatments">Treatments</a></p>\n<p><a href="/fees">Fees</a></p>\n<p><a href="tel:02036677070">020 3667 7070</a></p>\n<p><a href="https://uk.dentalhub.online/soe/new/London%20Dental%20Centre?pid=UKLZC01">Book Online Now</a></p>\n<p>Open Menu Close Menu</p>\n<p><a href="/"><img alt="London Dental Centre" src="https://thelondondentalcentre.co.uk/images.squarespace-cdn

In [18]:
!pip install beautifulsoup4

  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)


In [19]:
from bs4 import BeautifulSoup

In [27]:
elements = soup.find_all(['p', 'div', 'span','h1','h2','h3'])  # You can add more tags to this list
for elem in elements:
    print('-'*50)
    print(elem.get_text(strip=True))


--------------------------------------------------
By using this website, you agree to our use of cookies. We use cookies to provide you with a great experience and to help our website run effectively.
--------------------------------------------------
OK
--------------------------------------------------
0
--------------------------------------------------

--------------------------------------------------
Interest-Free Finance Available
--------------------------------------------------

--------------------------------------------------
About Us
--------------------------------------------------
Treatments
--------------------------------------------------
Fees
--------------------------------------------------
020 3667 7070
--------------------------------------------------
Book Online Now
--------------------------------------------------
Open Menu Close Menu
--------------------------------------------------

--------------------------------------------------
About Us
----------

'By using this website, you agree to our use of cookies. We use cookies to provide you with a great experience and to help our website run effectively.\nOK\n0\n\nInterest-Free Finance Available\n\nAbout Us\nTreatments\nFees\n020 3667 7070\nBook Online Now\nOpen Menu Close Menu\n\nAbout Us\nTreatments\nFees\n020 3667 7070\nBook Online Now\nOpen Menu Close Menu\nAbout Us\nTreatments\nFees\n020 3667 7070\nBook Online Now\n\nYour Trusted Dentists in the City of London\nWe’re dedicated to personalised care that makes you feel valued, using world-class techniques to enhance your smile and boost your confidence.\n% buffered00:00\nPlay\nVideo is not available or format is not supported. Try a different browser.\nContact Us\nBook Now\n\nWhere expertise meets elegance .\n\n\n\nWe all need a dentist. Picking the right one can be a major decision. Here’s why you should choose us .\nWe are committed to providing a smooth and relaxed dental experience.\nWe offer all types of dentistry including gene

In [155]:
import re

# def preprocess_content(content):
#     # Remove all menu items and similar patterns
#     content = re.sub(r'\s+\[.*?\]\(.*?\)', '', content)  # Remove patterns like [text](URL)
#     content = re.sub(r'\[Skip to the content\]\(.*?\)', '', content)  # Specific removal for "Skip to the content"
#     content = re.sub(r'\[.*?\]\(.*?\)', '', content)  # Generic removal for [text](URL)
#     content = re.sub(r'\s*Menu\s*', '', content)  # Remove "Menu"
#     content = re.sub(r'\s*Search\s*', '', content)  # Remove "Search"
#     content = re.sub(r'Categories\s*', '', content)  # Remove "Categories"

#     # Remove all URLs
#     content = re.sub(r'http\S+', '', content)

#     # Remove empty lines or lines with only whitespace
#     content = '\n'.join([line for line in content.splitlines() if line.strip()])

#     # Optionally, you could limit the text to a specific number of words

#     return content

In [156]:
with open("example.txt", "w") as file:
    file.write(html_content[0].page_content)

In [157]:

from bs4 import BeautifulSoup

In [28]:
def get_text_str(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    out_text = ""
    elements = soup.find_all(['p', 'div', 'span','h1','h2','h3'])  # You can add more tags to this list
    for elem in elements:
        
        out_text += ('-'*50)
        out_text += '\n'
        out_text += elem.get_text()
        out_text += '\n'
    return out_text

In [194]:
import re

def process_text(text):
    # Split the text into lines
    lines = text.splitlines()
    
    # Initialize variables
    processed_lines = []
    ignore_content = False
    merge_line = ""
    
    cookies_meta_found = False
    cookies_meta_line_position = 0

    for i, line in enumerate(lines):
        # Remove leading and trailing whitespace from the line
        line = line.strip()

        # Skip empty lines
        if not line:
            continue
        
        # Rule 1: Ignore lines starting with '#'
        if line.startswith('#'):
            continue
        
        # Rule 2: Ignore lines with '|'
        if '|' in line:
            continue
        
        # Rule 3: Ignore lines with 'copyright', 'Copy Right', or a copyright symbol
        if re.search(r'copyright|Copy Right|©', line, re.IGNORECASE):
            continue
        
        # Rule 4: Handle cookies/meta-data-related content
        if re.search(r'cookies?|meta[-_ ]data', line, re.IGNORECASE):
            if i < 25:
                # If found within the first 25 lines, ignore the line
                continue
            else:
                # If found after the 25th line, set the flag to ignore subsequent lines
                cookies_meta_found = True
                cookies_meta_line_position = i
                continue

        # If ignore_content flag is set and we are in the next line after the found keyword
        if cookies_meta_found and i > cookies_meta_line_position:
            continue
        
        # Rule 5: Treat content between '--------' lines as a single line
        if line.startswith('--------'):
            if merge_line:
                processed_lines.append(merge_line.strip())
                merge_line = ""
            continue
        else:
            merge_line += " " + line if merge_line else line
        
        # Additional Rule: Ignore lines containing "Open Menu", "Close Menu", "Privacy", or "Follow Us"
        if re.search(r'Open Menu|Close Menu|Privacy|Follow Us', line, re.IGNORECASE):
            continue
    
    # Add the last accumulated line if any
    if merge_line:
        processed_lines.append(merge_line.strip())

    # Apply further post-processing rules
    final_lines = []
    for line in processed_lines:
        # Rule 6: Remove lines that contain only numbers
        if line.isdigit():
            continue
        
        # Rule 7: Remove lines that have only two words or fewer
        if len(line.split()) <= 2:
            continue
        
        # Rule 8: Remove lines with all capital letters and less than 4 words
        if line.isupper() and len(line.split()) < 4:
            continue
        
        final_lines.append(line)

    # Join the final lines into a single string
    processed_text = '\n'.join(final_lines)
    processed_text = processed_text.replace("\\", "")
    return processed_text


In [195]:
import re

def clean_text(input_string):
    # Define the keywords to remove lines containing them
    keywords = ['Open Menu', 'Close Menu', 'Privacy', 'Term and Condition', 'T&C', 
                'Facebook', 'http', 'https', 'Insta', 'Twitter', 'LinkedIn', 'Snapchat', 'TikTok','field']

    # Split the input string into lines
    lines = input_string.splitlines()

    # Initialize a set to track unique lines
    unique_lines = set()

    # Define a regex pattern to detect lines with only numeric characters or less than 4 alphabetic characters
    alpha_numeric_pattern = re.compile(r'^[\d\s]*[a-zA-Z]{0,3}[\d\s]*$')

    # Define a regex pattern to identify numbers followed by words with no space, excluding ordinal numbers
    number_word_pattern = re.compile(r'(\d)(?!st|nd|rd|th)([a-zA-Z])')

    # Process each line
    cleaned_lines = []
    for line in lines:
        # Insert newline if a number is followed by a word without space, except for ordinal numbers
        line = number_word_pattern.sub(r'\1\n\2', line)

        # Check if line contains any of the keywords or matches the numeric pattern
        if (any(keyword.lower() in line.lower() for keyword in keywords) or
                alpha_numeric_pattern.match(line)):
            continue
        
        # Check for duplicates
        if line not in unique_lines:
            unique_lines.add(line)
            cleaned_lines.append(line)
    
    # Join the cleaned lines back into a string
    return '\n'.join(cleaned_lines)

In [196]:
def preproces_content_new(html_page_content):
    html = markdown.markdown(html_page_content)
    text_o = get_text_str(html)
    preprocessed_text = process_text(text_o)
    cleaned_text = clean_text(preprocessed_text)
    return cleaned_text

In [197]:
import re

In [198]:
out_text = preproces_content_new(html_content[0].page_content)

In [199]:
out_text

'Cart is empty\nWelcome To CyberCity\nWho We Are ?\nWelcome to Cyber City, your trusted destination for computer accessories and electronics located in the renowned Rex City Computer Market of Faisalabad. Since opening our doors in 1998, we have been dedicated to serving both retail and wholesale customers with a commitment to quality that has become synonymous with our name. Our reputation for reliability and exceptional customer service continues to drive our success.\nOver the years, we have continually expanded our product offerings to include everything from cutting-edge gaming equipment to essential networking solutions, storage devices, converters, and a comprehensive array of audio products. We maintain strong partnerships with leading manufacturers to ensure that our customers receive the finest products at competitive prices.\nProud of our prestigious history, Cyber City remains committed to innovation and excellence. We strive to stay at the forefront of technological advanc

In [200]:
with open("example.txt", "w") as file:
    file.write(out_text)

In [192]:
import re

In [28]:
def remove_all_repetitions(text):
    words = text.split()
    seen = set()
    result = []

    for word in words:
        if word not in seen:
            seen.add(word)
            result.append(word)

    return ' '.join(result)


In [29]:
import re


def extract_plain_text_from_markdown(markdown_content):
    # Remove image tags (Markdown and HTML)
    markdown_content = re.sub(r'!\[.*?\]\(.*?\)', '', markdown_content)
    markdown_content = re.sub(r'<img.*?>', '', markdown_content)
    
    # Remove links (Markdown and plain URLs)
    markdown_content = re.sub(r'\[.*?\]\(.*?\)', '', markdown_content)
    markdown_content = re.sub(r'\(https?://.*?\)', '', markdown_content)
    markdown_content = re.sub(r'https?://\S+|www\.\S+', '', markdown_content)
    
    # Remove any remaining inline links
    markdown_content = re.sub(r'\[.*?\]', '', markdown_content)
    
    # Remove HTML tags (including SVG)
    markdown_content = re.sub(r'<.*?>', '', markdown_content)
    
    # Remove all symbols except ., "", '', ?, and ,
    markdown_content = re.sub(r"[^a-zA-Z0-9\s.,\"'?']", '', markdown_content)
    
    # Remove any extra whitespace and newlines
    markdown_content = re.sub(r'\s+', ' ', markdown_content).strip()
    
    return markdown_content

In [30]:
cleaned_content = extract_plain_text_from_markdown(html_content[0].page_content)

In [31]:
cleaned_content = remove_all_repetitions(cleaned_content)

In [32]:
cleaned_content

'Solutions textgenerationinference documentation Flash Attention View all docsAWS Trainium InferentiaAccelerateAmazon SageMakerArgillaAutoTrainBitsandbytesChat UICompetitionsDataset viewerDatasetsDiffusersDistilabelEvaluateGoogle TPUsGradioHubHub Python LibraryHuggingface.jsInference API serverlessInference Endpoints dedicatedLeaderboardsOptimumPEFTSafetensorsSentence TransformersTRLTasksText Embeddings InferenceText Generation InferenceTokenizersTransformersTransformers.jstimm Search CtrlK main EN Getting started Tutorials Reference Conceptual Guides Join the Hugging Face community and get access to augmented experience Collaborate on models, datasets Spaces Faster examples with accelerated inference Switch between themes Scaling transformer architecture is heavily bottlenecked by selfattention mechanism, which has quadratic time memory complexity. Recent developments in accelerator hardware mainly focus enhancing compute capacities not transferring data hardware. This results attenti

In [195]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [196]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=8000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

In [197]:
texts = text_splitter.create_documents([cleaned_content])

In [198]:
len(texts)

2

In [199]:
from typing import Optional

In [200]:
from typing import List, Optional
from pydantic import BaseModel, Field, root_validator
from pydantic import BaseModel, Field, root_validator, ValidationError
from langchain.chains import LLMChain

In [201]:

# Define SentenceParser model
class SentenceParser(BaseModel):
    originalContent: str = Field(..., description="Original Sentence with no correction")
    correctedSentence: str = Field(..., description="Corrected Sentence in English UK. Enclosed corrections in <b></b>.")

    @root_validator(pre=True)
    def fill_missing_fields(cls, values):
        values['correctedSentence'] = values.get('correctedSentence', '')
        return values

# Define ResponseModel to include sentences
class ResponseModel(BaseModel):
    sentences: List[SentenceParser]

# Function to get the markdown content
def get_markdown_content(page_content):
    llm = ChatGroq(
            model="llama-3.1-70b-versatile",
            temperature=0,
            max_tokens=None,
            timeout=None,
            max_retries=3,
            # other params...
        )
    

    parser = PydanticOutputParser(pydantic_object=ResponseModel)

    prompt = PromptTemplate(
        template="""
        You are a smart assistant.
        You will be given a page-long document.
        You will return a list of sentences in the page.
        If there is any HTML content or non-textual content, you will remove it.
        First, you will clean the text by removing the images and links, and extract plain text.
        You will not expand any point or enhance anything. 
        If sentence contain the social media or web development related things like cookies etc you will remove it.
        Just return what is in the original text.
        Provide the English UK version of sentences, performing both spelling and grammar checks.
        Return the result as a JSON object with a single key "sentences" which contains a list of objects.
        Each object should have keys "originalContent" and "correctedSentence".

        \n\n{query}\n
        """,
        input_variables=["query"]
    )

    chain = LLMChain(llm=llm, prompt=prompt, output_parser=parser)

    result = chain({"query": page_content})

    if isinstance(result,dict):
        result = result['text']
        return result
    return result

/var/folders/gv/v0q4rb4n3p9_ydb839lgywph0000gq/T/ipykernel_8980/3567739227.py:6: PydanticDeprecatedSince20: Pydantic V1 style `@root_validator` validators are deprecated. You should migrate to Pydantic V2 style `@model_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  @root_validator(pre=True)


In [202]:
text_random = texts[0]

In [203]:
text_random

Document(page_content='Back to Top CS229 Gaussian Discriminant Analysis Generative Learning Algorithms Weve mainly looked at learning algorithms that model Pyx, the conditional distribution of y given x. For instance, logistic regression modeled Pyx as hxgTx where g is sigmoid function. In this section, well talk about a different type algorithm. Lets use binary classification problem motivation behind our discussion. Consider in which we want learn distinguish between malignant y1 and benign y0 tumors. Given training set, an algorithm like , or initially starts with randomly initialized parameters. Over course learning, performs gradient descent straight line hyperplane decision boundary evolves until you obtain separates positivenegative examples case, Then, classify new sample either benign, it checks on side falls in, makes its prediction accordingly. Theres class arent trying maximize likelihood looking both classes searching for separation boundary. Instead, these look one time. 

In [204]:
mkd_contents = get_markdown_content(text_random)

In [205]:
out_sentences = mkd_contents.dict().get('sentences')

/var/folders/gv/v0q4rb4n3p9_ydb839lgywph0000gq/T/ipykernel_8980/985300905.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  out_sentences = mkd_contents.dict().get('sentences')


In [206]:
out_sentences

[{'originalContent': 'Back to Top CS229 Gaussian Discriminant Analysis Generative Learning Algorithms Weve mainly looked at learning algorithms that model Pyx, the conditional distribution of y given x.',
  'correctedSentence': "Back to Top CS229 Gaussian Discriminant Analysis Generative Learning Algorithms We've mainly looked at learning algorithms that model Pyx, the conditional distribution of y given x."},
 {'originalContent': 'For instance, logistic regression modeled Pyx as hxgTx where g is sigmoid function.',
  'correctedSentence': 'For instance, logistic regression modelled Pyx as hxgTx where g is sigmoid function.'},
 {'originalContent': 'In this section, well talk about a different type algorithm.',
  'correctedSentence': "In this section, we'll talk about a different type of algorithm."},
 {'originalContent': 'Lets use binary classification problem motivation behind our discussion.',
  'correctedSentence': "Let's use binary classification problem motivation behind our discus

In [207]:
from difflib import ndiff

In [208]:
data = out_sentences

In [209]:
data

[{'originalContent': 'Back to Top CS229 Gaussian Discriminant Analysis Generative Learning Algorithms Weve mainly looked at learning algorithms that model Pyx, the conditional distribution of y given x.',
  'correctedSentence': "Back to Top CS229 Gaussian Discriminant Analysis Generative Learning Algorithms We've mainly looked at learning algorithms that model Pyx, the conditional distribution of y given x."},
 {'originalContent': 'For instance, logistic regression modeled Pyx as hxgTx where g is sigmoid function.',
  'correctedSentence': 'For instance, logistic regression modelled Pyx as hxgTx where g is sigmoid function.'},
 {'originalContent': 'In this section, well talk about a different type algorithm.',
  'correctedSentence': "In this section, we'll talk about a different type of algorithm."},
 {'originalContent': 'Lets use binary classification problem motivation behind our discussion.',
  'correctedSentence': "Let's use binary classification problem motivation behind our discus

In [210]:
from difflib import ndiff

def highlight_changes(original, corrected):
    diff = list(ndiff(original.split(), corrected.split()))
    result = []
    in_change = False

    for d in diff:
        if d.startswith('- '):
            continue
        elif d.startswith('+ '):
            if not in_change:
                result.append('<b>')
                in_change = True
            result.append(d[2:])
        elif d.startswith('  '):
            if in_change:
                result.append('</b>')
                in_change = False
            result.append(d[2:])
    
    if in_change:
        result.append('</b>')

    return ' '.join(result)

# Test
original_text = "This is the original text"
corrected_text = "This is the corrected text"
print(highlight_changes(original_text, corrected_text))

This is the <b> corrected </b> text


In [211]:
highlighted_sentences = [highlight_changes(pair['originalContent'], pair['correctedSentence']) for pair in data]
highlighted_sentences

["Back to Top CS229 Gaussian Discriminant Analysis Generative Learning Algorithms <b> We've </b> mainly looked at learning algorithms that model Pyx, the conditional distribution of y given x.",
 'For instance, logistic regression <b> modelled </b> Pyx as hxgTx where g is sigmoid function.',
 "In this section, <b> we'll </b> talk about a different type <b> of </b> algorithm.",
 "<b> Let's </b> use binary classification problem motivation behind our discussion.",
 'Consider in which we want <b> to </b> learn <b> to </b> distinguish between malignant y1 and benign y0 <b> tumours. </b>',
 'Given <b> a </b> training set, an algorithm like , or initially starts with randomly <b> initialised </b> parameters.',
 'Over <b> the </b> course <b> of </b> learning, <b> it </b> performs gradient descent <b> on a </b> straight line hyperplane decision <b> boundary, which </b> evolves until <b> it </b> separates <b> positive and negative examples. </b> Then, <b> it classifies a </b> new sample <b> as 

In [212]:
import pandas as pd

In [213]:
df = pd.DataFrame(out_sentences)

In [214]:
df

,originalContent,correctedSentence
0,Back to Top CS229 Gaussian Discriminant Analys...,Back to Top CS229 Gaussian Discriminant Analys...
1,"For instance, logistic regression modeled Pyx ...","For instance, logistic regression modelled Pyx..."
2,"In this section, well talk about a different t...","In this section, we'll talk about a different ..."
3,Lets use binary classification problem motivat...,Let's use binary classification problem motiva...
4,Consider in which we want learn distinguish be...,Consider in which we want to learn to distingu...
...,...,...
125,"3. converse, implication does Gaussian.","The converse is also true, and the implication..."
126,"3 2. stronger prove 2, noted property GDA.","3 implies 2, and this is a stronger result. We..."
127,"correct, better regression.","This is correct, and better than regression."
128,"Specifically, indeed asymptotically efficient.","Specifically, it is indeed asymptotically effi..."


In [215]:
df['changes'] = highlighted_sentences

In [216]:
filtered_df = df[df['changes'].str.contains('<b>')]

In [217]:
def normalize_and_filter(df, column_name):
    # Create a temporary column in lowercase for normalization and matching
    df['temp_lower'] = df[column_name].str.lower()
    
    # Pattern to match social media names, specific extensions, and keywords like 'cookies'
    pattern = r'\b(company|facebook|twitter|instagram|linkedin|youtube|\.com|\.ad|\.gov|\.uk|\.php|\.asp|cookies|cookies|statistical|tracking|ads|script|event|policy|Cloudflare|cookie|cfduid|llc|party)\b'
    
    # Filter rows where the temporary lowercase column does not contain any of the specified patterns
    filtered_df = df[~df['temp_lower'].str.contains(pattern, flags=re.IGNORECASE, regex=True)]
    
    # Drop the temporary column
    filtered_df.drop('temp_lower', axis=1, inplace=True)
    
    return filtered_df

In [218]:
filtered_df = normalize_and_filter(df, 'changes')

/var/folders/gv/v0q4rb4n3p9_ydb839lgywph0000gq/T/ipykernel_8980/3883782903.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_df = df[~df['temp_lower'].str.contains(pattern, flags=re.IGNORECASE, regex=True)]


In [219]:
filtered_df.iloc[0]['changes']


"Back to Top CS229 Gaussian Discriminant Analysis Generative Learning Algorithms <b> We've </b> mainly looked at learning algorithms that model Pyx, the conditional distribution of y given x."

In [220]:
filtered_df.iloc[0]['originalContent']

'Back to Top CS229 Gaussian Discriminant Analysis Generative Learning Algorithms Weve mainly looked at learning algorithms that model Pyx, the conditional distribution of y given x.'

In [221]:
df.to_json()

'{"originalContent":{"0":"Back to Top CS229 Gaussian Discriminant Analysis Generative Learning Algorithms Weve mainly looked at learning algorithms that model Pyx, the conditional distribution of y given x.","1":"For instance, logistic regression modeled Pyx as hxgTx where g is sigmoid function.","2":"In this section, well talk about a different type algorithm.","3":"Lets use binary classification problem motivation behind our discussion.","4":"Consider in which we want learn distinguish between malignant y1 and benign y0 tumors.","5":"Given training set, an algorithm like , or initially starts with randomly initialized parameters.","6":"Over course learning, performs gradient descent straight line hyperplane decision boundary evolves until you obtain separates positivenegative examples case, Then, classify new sample either benign, it checks on side falls in, makes its prediction accordingly.","7":"Theres class arent trying maximize likelihood looking both classes searching for separa

In [222]:
df_dict = df.to_dict(orient='records')

In [223]:
data = df_dict

In [224]:
filtered_df = df[df['changes'].str.contains('<b>')]

In [226]:
filtered_df.to_csv('a_data.csv', index=False)

In [227]:
sentences = list(filtered_df['originalContent'])

In [228]:
caution_word_list = ["Best", "Specialist", "Specialised", "Finest", "Most experienced",
                                "Superior", "Principle", "Expert", "Amazing", "Speciality",
                                "leader", "leaders", "service", "implantologist"
            ]

In [229]:


# Function to add <b></b> around caution words
def highlight_caution_words(sentence):
    pattern = re.compile(r'\b(' + '|'.join(re.escape(word) for word in caution_word_list) + r')\b', re.IGNORECASE)
    highlighted_sentence = pattern.sub(r'<b>\1</b>', sentence)
    if highlighted_sentence != sentence:
        return highlighted_sentence
    return None

# Apply the function to all sentences and filter out None results
highlighted_sentences = [highlight_caution_words(sentence) for sentence in sentences if highlight_caution_words(sentence)]

# Print the results
for sentence in highlighted_sentences:
    print(sentence)

discuss <b>superior</b> viceversa.
